In [1]:
import torch
import pandas as pd
import torch.nn as nn
from torch.utils import data
import numpy as np

import os

In [2]:
DATA_DIR = '/Users/aplle/Code/MathModeling/SC/dataset1'
MODEL_DIR = '/Users/aplle/Code/MathModeling/SC/models'

In [3]:
train_df = pd.read_csv(f"{DATA_DIR}/train.csv")
eval_df = pd.read_csv(f"{DATA_DIR}/eval.csv")
test_df = pd.read_csv(f"{DATA_DIR}/test.csv")

POWER_DIVISOR = 1000.0
YEAR_MIN = 2016
YEAR_MAX = 2018

def is_leap(yr:int):
    if yr % 400 == 0:
        return True
    elif yr % 100 == 0:
        return False
    elif yr % 4 == 0:
        return True
    return False

def get_ndays(yr:int):
    return 366 if is_leap(yr) else 365

def preprocess_df(df):
    df.dropna(subset=['Power (kW)'], inplace=True)
    df['Power (kW)'] = df['Power (kW)'] / POWER_DIVISOR
    df.rename(columns={'Power (kW)': 'Power'}, inplace=True)

    df['Days_from_NYD'] = df['Days_from_NYD'] / df['Year'].apply(get_ndays)

    df['Year'] = df['Year'] - YEAR_MIN
    df['Year'] = df['Year'] / (YEAR_MAX - YEAR_MIN)

    df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S')
    df['Time'] = df['Time'].dt.hour * 60 + df['Time'].dt.minute
    df['Time'] = df['Time'] / (24 * 60)

    df.drop(['Day', 'Span'], axis=1, inplace=True)

    df['Month'] = df['Month'] / 12.0

    df['Weekday'] = df['Weekday'] / 6.0

    df['Region'] = df['Region'].map({
        'Commercial': 0,
        'Office': 1,
        'Public': 2,
        'Residential': 3
    })

    return df

train_df = preprocess_df(train_df)
_train_columns = train_df.columns.copy()

eval_df = preprocess_df(eval_df)
eval_df = eval_df.reindex(columns=_train_columns, fill_value=0)

test_df = preprocess_df(test_df)
test_df = test_df.reindex(columns=_train_columns, fill_value=0)

In [ ]:
lasso_model = nn.Linear(in_features=len(train_df.columns)-1, out_features=1, bias=True)
criterion = nn.MSELoss() + nn.L1Loss()



# ABANDONED CODE